In this notebook, I have compared the results of content based and collaborative recommendation systems. I have used some of the ideas from this kernel:-
https://www.kaggle.com/fabiendaniel/film-recommendation-engine
This notebook can be used as a template for recommendation engine implementation in Python.
I have used the Amazon Book Review Dataset that can be found here.
http://jmcauley.ucsd.edu/data/amazon/

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import timeit
import time
import resource

In [5]:
path = r"C:\Users\Neha\Project_Files\Old_Ipynb_Files\Books_5.json"
#c = pd.read_json("/home/science/small",orient="columns")

#customers_json = pd.read_json('small')

In [ ]:
def parse(path):
  g = open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

#def f(row):
    #if (row.overall == 4 or row.overall == 5):
        #val = 3
    #elif (row.overall ==3):
        #val = 2
    #else:
        #val = 1
    #return val


df = getDF(path)
df.info()

In [ ]:
df = df.drop('reviewerName', 1)
df = df.drop('helpful', 1)
#df = df.drop('summary', 1)
df = df.drop('unixReviewTime', 1)
#df = df.drop('reviewerID', 1)
#df = df.drop('asin', 1)
df = df.drop('reviewTime', 1)
#df['rating'] = df.apply(f, axis=1)
#df = df.drop('overall', 1)
df

In [ ]:
### Collaborative Filtering
ratings_matrix = df.pivot_table(index=['asin'],columns=['reviewerID'],values='overall').reset_index(drop=True)
ratings_matrix.fillna( 0, inplace = True )
movie_similarity=cosine_similarity(ratings_matrix)
np.fill_diagonal( movie_similarity, 0 ) 
movie_similarity

In [49]:
#### Sentiment Analysis
trainX,testX,trainY,testY = train_test_split(df['reviewText'],df['rating'], test_size = 0.2, random_state = 10)
cv = CountVectorizer()

start_mem = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
t = time.time()
cv.fit(trainX)
trainX_cv = cv.transform(trainX)
testX_cv = cv.transform(testX)
model = MultinomialNB()
model.fit(trainX_cv,trainY)

elapsed_time = time.time() - t
mem_used = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss - start_mem

print("time taken by train function",(elapsed_time))
print("memory used by train function",(mem_used))
cv_score = accuracy_score(testY,model.predict(testX_cv))
print("cv score", (cv_score))

time taken by train function 471.379958152771
memory used by train function 1033868


In [50]:
##### Content Based Recommendation

0.80130000000000001